In [ ]:
library(dplyr)
library(lubridate)
library(R.matlab)
library(rdbnomics)
library(tibble)
library(tidyr)

In [ ]:
gdp_values <- rdb(ids="INSEE/CNT-2010-PIB-EQB-RF/T.CNT-EQUILIBRE_PIB.SO.PIB.SO.VALEUR_ABSOLUE.FE.V.EUROS.CVS-CJO") %>%
add_column(var="gdp_values")

In [ ]:
expenditures_values <-
rdb(ids="INSEE/CNT-2010-CSI/T.CNT-OPERATIONS_SECTEURS_INST.S13.E.OTE.VALEUR_ABSOLUE.FE.EUROS.CVS-CJO") %>%
add_column(var="expenditures_values")

In [ ]:
debt_gdp <-
rdb(ids="INSEE/DETTE-TRIM-APU/T.DETTE_MAASTRICHT.S13.F.PROPORTION.FE.POURCENT.BRUT.SO") %>%
add_column(var="debt_gdp") %>%
mutate(value=value/100)

In [ ]:
df <-
bind_rows(gdp_values,expenditures_values,debt_gdp) %>%
select(period,value,var) %>%
spread(var,value) %>%
mutate(expenditures_gdp=expenditures_values/gdp_values) %>%
select(period,expenditures_gdp,debt_gdp) %>%
filter(year(period)>=1996 & year(period)<=2017) %>%
mutate(period=paste(year(period),quarter(period),sep="Q"))

colnames(df)[1] <- ""

write.csv(df,"data.csv",row.names = F)

In [ ]:
system2(
    'octave',
    args = c('--eval', '"addpath /usr/local/lib/dynare/matlab/; dynare model.mod"', '--no-window-system'),
    stdout = "stdout.txt",
    stderr = "stderr.txt",
    stdin = "",
    input = NULL,
    env = character(),
    wait = TRUE,
)
# The output of this command is written in files `stderr.txt` and `stdout.txt`.
# Dynare use the model `model.mod` and generate the files beginning with `model`.

In [ ]:
user <- Sys.getenv(c('JUPYTERHUB_USER'))
output <- system2(
    'jupyter',
    args = c('notebook', 'list'),
    stdout = TRUE,
    stderr = TRUE,
    stdin = "",
    input = NULL,
    env = character(),
    wait = TRUE,
)
match <- regexpr("(?<=token=)[^ ]*", output, perl=TRUE)
token <- regmatches(output, match)[1]
model_url <- paste0('https://hub.mybinder.org/user/', user, '/files/home/jovyan/model.m?token=', token)
shiny_url <- paste0('https://shiny.nomics.world/toto/?url=', URLencode(model_url, reserved=TRUE))
IRdisplay::display_html(paste0('<iframe src="', shiny_url, '" height="600" width="800">Votre navigateur n\'affiche pas cette iframe</iframe>'))